In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import tqdm
import cv2
from PIL import Image

import numpy as np
import os
import pandas as pd
import shutil

from shutil import unpack_archive
from subprocess import check_output

In [ ]:
df=pd.read_csv("../input/sartorius-cell-instance-segmentation/train.csv")

In [ ]:
df_files=df[["id","cell_type"]]
df_files=df_files.drop_duplicates(subset="id").reset_index(drop=True)
df_files.head()

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=221)

for fold, (_, val_idx) in enumerate(skf.split(X=df_files, y=df_files['cell_type']), 1):
    df_files.loc[val_idx, 'fold'] = fold
    
df_files['fold'] = df_files['fold'].astype(np.uint8)

In [ ]:
df_files.head()

In [ ]:
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)

In [ ]:
shutil.os.mkdir('tenfolds')

In [ ]:
for fold in range(1,11):
    sum = 0
    location=f'tenfolds/fold_{fold}/train/masks/'
    os.makedirs(f"tenfolds/fold_{fold}/train/masks",exist_ok=True)
    os.makedirs(f"tenfolds/fold_{fold}/train/input",exist_ok=True)
    file_names_train=df_files.loc[df_files.fold!=fold]["id"].unique()
    for file in tqdm.tqdm(file_names_train):
        main_image = np.zeros(520*704, dtype=np.float32)
        main_image = main_image.reshape((520,704))
        #main_image = np.zeros([520,704])
        get_all_codes = df[df['id'] == file]
        get_all_codes = get_all_codes.reset_index()
        for d in range(len(get_all_codes)):
            #print(get_all_codes.loc[d]['id'])
            try:
                im = rle_decode(get_all_codes.loc[d]['annotation'],(get_all_codes.loc[d]['height'],get_all_codes.loc[d]['width']))
                main_image = cv2.bitwise_or(main_image,im)
            except Exception as e:
                sum+=1
        # save masks as png image
        img = Image.open('../input/sartorius-cell-instance-segmentation/train/' + file +'.png')
        img.save(f'tenfolds/fold_{fold}/train/input/'+file+'.tiff')
        cv2.imwrite(location+file+'.tiff',main_image)
    print(sum)

In [ ]:
for fold in range(1,11):
    sum = 0
    location=f'tenfolds/fold_{fold}/val/masks/'
    os.makedirs(f"tenfolds/fold_{fold}/val/masks",exist_ok=True)
    os.makedirs(f"tenfolds/fold_{fold}/val/input",exist_ok=True)
    file_names_val=df_files.loc[df_files.fold==fold]["id"].unique()
    for file in tqdm.tqdm(file_names_val):
        main_image = np.zeros(520*704, dtype=np.float32)
        main_image = main_image.reshape((520,704))
        #main_image = np.zeros([520,704])
        get_all_codes = df[df['id'] == file]
        get_all_codes = get_all_codes.reset_index()
        for d in range(len(get_all_codes)):
            #print(get_all_codes.loc[d]['id'])
            try:
                im = rle_decode(get_all_codes.loc[d]['annotation'],(get_all_codes.loc[d]['height'],get_all_codes.loc[d]['width']))
                main_image = cv2.bitwise_or(main_image,im)
            except Exception as e:
                sum+=1
        # save masks as png image
        img = Image.open('../input/sartorius-cell-instance-segmentation/train/' + file +'.png')
        img.save(f'tenfolds/fold_{fold}/val/input/'+file+'.tiff')
        cv2.imwrite(location+file+'.tiff',main_image)
    print(sum)

In [ ]:
!zip -r -q tensfolds.zip tenfolds

In [ ]:
!rm -r tenfolds